# Assignment 4.1: Tilt Alarm System

## Part A4.1

### Buzzer sensor module used: KY006 (Passive) or KY012- (Active)

#### Include placeholder

In [3]:
import time
from pynq.lib import Pmod_IO
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")
import socket
import multiprocessing

In [4]:
FAILURE = -1
SUCCESS = 0
SERVER_FREQ = 2500

In [5]:
#creat Pmod_IO object to be used for server. Wired to + of buzzer sensor to PMODB
gpio_pin_1 = Pmod_IO(base.PMODB,1,'out')
#create Pmod_IO object to be used for client. Wired to + of buzzer sensor to PMODA
gpio_pin_2 = Pmod_IO(base.PMODA,1,'out')

In [6]:
#store these pins in a dictionary to easily refrence based on server client
pins = {
    'server'   : gpio_pin_1, 
    'client'   : gpio_pin_2,
}

In [7]:
# definition to turn buzzer off
def turn_buzzer_off(index):
    pins[index].write(0)

In [8]:
# def function to test buzzer
def turn_buzzer_on(index):
    pins[index].write(1)

In [9]:
# def function to turn buzzer on for particular frequency
def turn_alarm_on(freq, index, play_time):
    if freq <= 0:
        print("frequency should be > 0")
        return FAILURE
    if index not in pins:
        print("index passed does not exist {}".format(index))
        return FAILURE
    period = round(1.0/freq,6)
    cycles = round(play_time/period)
    while cycles:
        turn_buzzer_on(index)
        time.sleep((1/(2*freq)))
        turn_buzzer_off(index)
        time.sleep((1/(2*freq)))
        cycles -=1
    
    return SUCCESS

In [10]:
freq = 25000 #Hz
index = 'server'
count = 0
turn_alarm_on(freq, index, 0.5)
count +=1

In [11]:
from enum import Enum
class Button_State(Enum):
    Connect = 0
    Alarm = 1
    Disconnect = 2

In [12]:
address = ('localhost', 6000)     # family is deduced to be 'AF_INET'

In [13]:
def server():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # 1: Bind the socket to the pynq board <CLIENT-IP> at port <LISTENING-PORT>
    # bind the socket to a public host, and a well-known port
    sock.bind(("127.0.0.1", 33334))
    
    #Always be a listener
    sock.listen(1)
    cond = False;
    while True:
        # 2: Accept connections
        clientsocket, addr = sock.accept()
        print ('Got connection from', addr )
        
        # 3: Receive bytes from the connection 
        msg = clientsocket.recv(1024).decode()
        print ("Message received {}\n".format(msg))
        
        if msg == "alarm":
            print ("ServermMessage {}\n".format(msg))
            cond = True
         
        if msg == "disconnect":
            print ("Server Message {}\n".format(msg))
            cond = False
           
        if cond == True:
            print ("Server Buzzer On\n")
            turn_alarm_on(SERVER_FREQ, 'server', 0.3)

In [14]:
def client(btns):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect(('127.0.0.1', 33334))
    Button_cur_State = Button_State.Connect
    
    cond = True
    
    while cond:
        #time.sleep(0.01)
        if btns.read() == 2:
            if Button_cur_State != Button_State.Alarm:
                Button_cur_State = Button_State.Alarm
                print("Client: alarm Button state = {}\n".format(Button_cur_State))
                sock.send("alarm".encode())
        if btns.read() == 4:
            if Button_cur_State != Button_State.Disconnect:
                print("Client: disconnect")
                Button_cur_State = Button_State.Disconnect
                print("Client: Disconnect Button state = {}\n".format(Button_cur_State))
                sock.send("disconnect".encode())
                cond = False
    sock.close()

In [15]:
procs = [] # a future list of all our processes


In [16]:
import threading
import os
def main():
    btns = base.btns_gpio
    
    #create a asyncio loop event     
    # Launch process1 on CPU0
    p1_start = time.time()
    p1 = multiprocessing.Process(target=server, args=()) 
    os.system("taskset -p -c {} {}".format(0, p1.pid)) 
    p1.start() # start the process
    procs.append(p1)

    # Launch process2 on CPU1
    cond = True
    while cond:
        time.sleep(0.01)
        if btns.read() == 1:
            cond = False
            print("Creating client")
            p2_start = time.time()
            p2 = multiprocessing.Process(target=client, args=(btns,)) 
            os.system("taskset -p -c {} {}".format(1, p2.pid))
            p2.start() # start the process
            procs.append(p2)
    
    p1Name = p1.name # get process1 name
    p2Name = p2.name # get process2 name
    
    p2.join() # wait for process2 to finish
    print('Process 2 with name, {}, is finished'.format(p2Name))
    
    # Here we wait for process1 to finish then wait for process2 to finish
    p1.join() # wait for process1 to finish
    print('Process 1 with name, {}, is finished'.format(p1Name))

In [ ]:
#if __name__ == '__main__':
main()

Creating client
Got connection from ('127.0.0.1', 58372)
Client: alarm Button state = Button_State.Alarm

Message received alarm

ServermMessage alarm

Server Buzzer On

Client: disconnect
Client: Disconnect Button state = Button_State.Disconnect

Process 2 with name, Process-2, is finished
